In [64]:
import pandas as pd
from tqdm import tqdm

books = pd.read_csv("books_with_categories.csv")

In [65]:
#let's finding a fine tune model for emotion classification
#opensource model from hugging face
# we are treating sentiment analysis as a text classification problem
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base",top_k= None, device= "mps")
classifier("I love this!")


Device set to use mps


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764600355178118},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [66]:
#2 options either to get sentiment classifier to predict overall emotions for the whole descriptio
# we can brake it down in individual sentences and try to predict the sentiment of each  of these
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [67]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548413634300232},
  {'label': 'neutral', 'score': 0.1698521375656128},
  {'label': 'sadness', 'score': 0.11640870571136475},
  {'label': 'surprise', 'score': 0.020700665190815926},
  {'label': 'disgust', 'score': 0.01910073310136795},
  {'label': 'joy', 'score': 0.015161280520260334},
  {'label': 'anger', 'score': 0.0039351508021354675}]]

In [68]:
#split the description down in sentences and classify them
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296023964881897},
  {'label': 'neutral', 'score': 0.14038589596748352},
  {'label': 'fear', 'score': 0.06816217303276062},
  {'label': 'joy', 'score': 0.04794251546263695},
  {'label': 'anger', 'score': 0.009156357496976852},
  {'label': 'disgust', 'score': 0.002628477755934},
  {'label': 'sadness', 'score': 0.0021221647039055824}],
 [{'label': 'neutral', 'score': 0.4493715167045593},
  {'label': 'disgust', 'score': 0.27359068393707275},
  {'label': 'joy', 'score': 0.10908294469118118},
  {'label': 'sadness', 'score': 0.09362741559743881},
  {'label': 'anger', 'score': 0.040478210896253586},
  {'label': 'surprise', 'score': 0.026970192790031433},
  {'label': 'fear', 'score': 0.006879056338220835}],
 [{'label': 'neutral', 'score': 0.6462160348892212},
  {'label': 'sadness', 'score': 0.2427334487438202},
  {'label': 'disgust', 'score': 0.0434226393699646},
  {'label': 'surprise', 'score': 0.02830049954354763},
  {'label': 'joy', 'score': 0.014211451634

In [69]:
#compare the prediction to the sentence to see if it's giving right
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [70]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [71]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296023964881897},
 {'label': 'neutral', 'score': 0.14038589596748352},
 {'label': 'fear', 'score': 0.06816217303276062},
 {'label': 'joy', 'score': 0.04794251546263695},
 {'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.002628477755934},
 {'label': 'sadness', 'score': 0.0021221647039055824}]

In [72]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [73]:
predictions[3]

[{'label': 'fear', 'score': 0.9281686544418335},
 {'label': 'anger', 'score': 0.03219062462449074},
 {'label': 'neutral', 'score': 0.012808652594685555},
 {'label': 'sadness', 'score': 0.008756848983466625},
 {'label': 'surprise', 'score': 0.008597859181463718},
 {'label': 'disgust', 'score': 0.008431786671280861},
 {'label': 'joy', 'score': 0.0010455807205289602}]

In [74]:
#for each book have the separate column for the which of the 7 emotions classes
#instead of assigning category we take the highest probability from across the whole description for that particular sentiment
# process the outpot so gives the maximum score for each of this labels
sorted(predictions[0],key = lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.002628477755934},
 {'label': 'fear', 'score': 0.06816217303276062},
 {'label': 'joy', 'score': 0.04794251546263695},
 {'label': 'neutral', 'score': 0.14038589596748352},
 {'label': 'sadness', 'score': 0.0021221647039055824},
 {'label': 'surprise', 'score': 0.7296023964881897}]

In [75]:
#extract max emotion probability for each emotion for each description
import numpy as np
#the key for our dictionary
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
#empty list for isbn because we will later create a dataframe and merge it into the dataframe
isbn =[]
#emotion score dictionary that contains all the score  for each single diescription of all these labels
emotion_scores = {label: [] for label in emotion_labels}
#extract the maximum emotion score from all description
def calculate_max_emotion_score(predictions):
    #designed to hold all the predictions for a particular description
    per_emotion_score = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key = lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
         #we loop over that and for every emotion we extract the score and we append it using the correct label
         per_emotion_score[label].append(sorted_predictions[index]["score"])
         #we append this to the dictionary comprehension and than we take the maximum score for each of the
    return {label: np.max(scores) for label,scores in per_emotion_score.items()}

In [78]:
#each description is a dictionary containing the maximum probability for each of the diffrent emotion labels
#now we need to apply  this function for each of the books
for i in range(10):
    #we take the isbn 13 and we appent it to the list
    isbn.append(books["isbn13"][i])
    #we take the description we split into individual sentences and save it into sentences
    sentence = books["description"][i].split(".")
    #we pass that variable into emotions classifier
    predictions = classifier(sentence)
    #we pass the predictions in max score  function and than we take the  result in dictionary that is returned by that function and we pend it to that emotion score dictionary
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [79]:
emotion_scores

{'anger': [0.06413353234529495,
  0.06413353234529495,
  0.6126179099082947,
  0.06413353234529495,
  0.35148343443870544,
  0.08141229301691055,
  0.2322252243757248,
  0.5381841659545898,
  0.06413353234529495,
  0.3006708025932312,
  0.06413353234529495,
  0.06413353234529495,
  0.6126179099082947,
  0.06413353234529495,
  0.35148343443870544,
  0.08141229301691055,
  0.2322252243757248,
  0.5381841659545898,
  0.06413353234529495,
  0.3006708025932312,
  0.06413353234529495],
 'disgust': [0.27359068393707275,
  0.27359068393707275,
  0.34828516840934753,
  0.1040065586566925,
  0.15072256326675415,
  0.1844949573278427,
  0.7271744608879089,
  0.15585489571094513,
  0.1040065586566925,
  0.27948132157325745,
  0.17792607843875885,
  0.27359068393707275,
  0.34828516840934753,
  0.1040065586566925,
  0.15072256326675415,
  0.1844949573278427,
  0.7271744608879089,
  0.15585489571094513,
  0.1040065586566925,
  0.27948132157325745,
  0.17792607843875885],
 'fear': [0.9281686544418335

In [83]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
#empty list for isbn because we will later create a dataframe and merge it into the dataframe
isbn =[]
#emotion score dictionary that contains all the score  for each single diescription of all these labels
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    #we take the isbn 13 and we appent it to the list
    isbn.append(books["isbn13"][i])
    #we take the description we split into individual sentences and save it into sentences
    sentence = books["description"][i].split(".")
    #we pass that variable into emotions classifier
    predictions = classifier(sentence)
    #we pass the predictions in max score  function and than we take the  result in dictionary that is returned by that function and we pend it to that emotion score dictionary
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])


100%|██████████| 5197/5197 [03:21<00:00, 25.74it/s]


In [84]:
#let's pass the results of the dictionary into the pandas dataframe
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn


In [86]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,9780002005883
1,0.612618,0.348285,0.942528,0.704422,0.887940,0.111690,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,9780006178736
3,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,9780006280934
...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255169,0.853722,0.980877,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,9788173031014
5194,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,9788179921623
5195,0.064134,0.104007,0.459269,0.759455,0.951104,0.368110,9788185300535


In [87]:
#merge into the books dataframe
books = pd.merge(books, emotions_df, on="isbn13")

In [88]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704422,0.887940,0.111690
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255169,0.853722,0.980877
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759455,0.951104,0.368110


In [89]:
books.to_csv("books_with_emotions.csv", index=False)